In [ ]:
import os
from utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology, grainGenerate
# from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset
from skimage import transform
from skimage import io, filters, util
import numpy as np
from skimage.color import rgb2gray

from tqdm.notebook import tqdm
import glob
from matplotlib import pyplot as plt
from skimage.morphology import disk, skeletonize, ball
import json

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

In [ ]:
def mixer_tranform(image):
    img = rgb2gray(image) * 255
    # global_thresh = filters.threshold_otsu(img)
    global_thresh = 67
    bin_image = img > global_thresh
    bin_image = bin_image.astype(np.uint8) * 255

    preproc_image = grainPreprocess.image_preprocess(bin_image)
    return preproc_image

## Предобработка всех фотографий

In [ ]:
# images_dir='../data/dataset_4x/'
# images_dir='images/artif/'
# save_name='images_4x_(512,512,1).npy'
# save_name='artif.npy'

# images_dir = '../data/dataset_sub_base_left/'
# images_dir = 'data/artif/mixer/'
# images_dir = 'data/artif_31.01/'
# save_name = '(1500,1500,1)_sub_base_left'
# save_name = '7_classes_mixer_preprocess'

images_dir = '../ml/vqvae2_rosalinity/latent_diff_dataset/'
# save_name = 'bc_left_(1536,1536,1)_preprocess'
save_name = 'latent_generated_(768,768,1)_preprocess'

all_images = grainPreprocess.read_preprocess_data(images_dir=images_dir,
                                                  save_name=save_name,
                                                  max_images_num_per_class=None,
                                                  preprocess_transform=None,
                                                  save=True,
                                                  crop_bottom=False,
                                                  resize_shape=None
                                                  # resize_shape=(1500, 1500, 1)
                                                  )

## Обработка каждой фотографии

In [ ]:
folder_path = '../datasets/original/o_bc_left_4x_768_360'
new_folder_path = '../datasets/original/o_bc_left_4x_768_360_median'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)

### Медианный фильтр + отсу

In [ ]:
for i, folder in tqdm(enumerate(folders)):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in tqdm(enumerate(images_names)):
        if 'hdr' not in name:
            image = io.imread(folder_path + '/' + folder + '/' + name)
            # image_combined = grainPreprocess.combine(image, h=144,k=1)
            height, width = image.shape
            img1 = image[:height // 2, :width // 2]
            img2 = image[:height // 2, width // 2:]
            img3 = image[height // 2:, :width // 2]
            img4 = image[height // 2:, width // 2:]
            # image_resized = transform.resize(image_combined, (512,512),preserve_range=True).astype(np.uint8)

            # io.imsave(new_folder_path + '/' + folder + '/' + name , image_combined)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_1' + name[-5:], img1)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_2' + name[-5:], img2)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_3' + name[-5:], img3)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_4' + name[-5:], img4)

### деление на 4 части

In [ ]:
for i, folder in tqdm(enumerate(folders)):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in tqdm(enumerate(images_names)):
        image = io.imread(folder_path + '/' + folder + '/' + name)
        
        unsigned_image = util.img_as_ubyte(image)
        if len(unsigned_image.shape) < 3:
            unsigned_image = unsigned_image[..., np.newaxis]
            
        denoised = filters.rank.median(unsigned_image, ball(3))
        
        # global_thresh = filters.threshold_otsu(denoised)
        # binary_global = denoised > global_thresh
        img=denoised.astype('uint8')
        
        io.imsave(new_folder_path + '/' + folder + '/' + name, img[:,:,0])


In [ ]:
folder_path = '../datasets/original/o_bc_left'
new_folder_path = '../datasets/original/o_bc_left_9x_512'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)
        
for i, folder in enumerate(folders):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in enumerate(images_names):
        if 'hdr' not in name:
            image = io.imread(folder_path + '/' + folder + '/' + name)
            # image_combined = grainPreprocess.combine(image, h=144,k=1)
            height, width = image.shape
            img1 = image[:height // 3, :width // 3]
            img2 = image[:height // 3, width // 3: width * 2 // 3]
            img3 = image[:height // 3:, width * 2 // 3:]
            img4 = image[height // 3:height * 2 // 3, :width // 3]
            img5 = image[height // 3:height * 2 // 3, width // 3: width * 2 // 3]
            img6 = image[height // 3:height * 2 // 3:, width * 2 // 3:]
            img7 = image[height * 2 // 3:, :width // 3]
            img8 = image[height * 2 // 3:, width // 3: width * 2 // 3]
            img9 = image[height * 2 // 3::, width * 2 // 3:]

            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_1' + name[-5:], img1)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_2' + name[-5:], img2)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_3' + name[-5:], img3)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_4' + name[-5:], img4)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_5' + name[-5:], img5)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_6' + name[-5:], img6)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_7' + name[-5:], img7)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_8' + name[-5:], img8)
            io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_part_9' + name[-5:], img9)


### Повороты

In [ ]:
folder_path = '../datasets/original/o_bc_left_9x_512'
new_folder_path = '../datasets/original/o_bc_left_9x_512_360'

folders = os.listdir(folder_path)

# создает папку для обработанных фотографий
if not os.path.exists(new_folder_path):
    os.mkdir(new_folder_path)

# создает папку для каждого класса фотографий
for folder in folders:
    if not os.path.exists(new_folder_path + '/' + folder):
        os.mkdir(new_folder_path + '/' + folder)

for i, folder in enumerate(folders):
    images_names = os.listdir(folder_path + '/' + folder)
    for i, name in enumerate(images_names):

        image = io.imread(folder_path + '/' + folder + '/' + name)

        height, width = image.shape
        img1 = image
        img2 = transform.rotate(image, 90)*255
        img3 = transform.rotate(image, 180)*255
        img4 = transform.rotate(image, 270)*255
        
        img2=img2.astype(np.uint8)
        img3=img3.astype(np.uint8)
        img4=img4.astype(np.uint8)

        io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_0' + name[-5:], img1)
        io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_90' + name[-5:], img2)
        io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_180' + name[-5:], img3)
        io.imsave(new_folder_path + '/' + folder + '/' + name[:-5] + '_angle_270' + name[-5:], img4)